In [1]:
import requests
from bs4 import BeautifulSoup
import torch


class LLMBrowser:
    def __init__(self, model, tokenizer, translator, max_new_tokens: int = 100, device: str = "cuda"):
        self.device = device
        self.model = model
        self.tokenizer = tokenizer
        self.translator = translator
        self.max_new_tokens = max_new_tokens
        
    def __call__(self, request: str):
        en_request = self.in_translate(request)
        q = "+".join(en_request.split())
        r = requests.get(f"https://www.google.com/search?q={q}")
        soup = BeautifulSoup(r.text, features="html.parser")

        soup = self.del_script_and_style(soup)
        doc = self.extract_text(soup)

        en_answer = self.generate(request, doc)
        ru_answer = self.out_translate(en_answer)
        return ru_answer

    def del_script_and_style(self, soup):
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        
        return soup

    def in_translate(self, text):
        source_lang = self.translator.detect_lang(text)
        return self.translator.translate(text, source_lang, "en")

    def out_translate(self, text):
        source_lang = self.translator.detect_lang(text)
        return self.translator.translate(text, "en", "ru")
        
    def extract_text(self, soup):
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        
        return text

    @torch.inference_mode()
    def generate(self, request, doc):

        content = f"Hey, Mistral! You now live in Russia, therefore answer all questions inconntext of Russia, Moscow only. Please answer the following question: {request}\n\n using this document: {doc}"
        
        messages = [
            {"role": "user", "content": content,}
        ]
        
        encodeds = self.tokenizer.apply_chat_template(messages, return_tensors="pt")
        model_inputs = encodeds.to(self.device)
        
        generated_ids = model.generate(model_inputs, max_new_tokens=self.max_new_tokens, do_sample=True)
        decoded = self.tokenizer.batch_decode(generated_ids)
        
        
        torch.cuda.empty_cache()
    
        return decoded[0][len(content) + len("[INST]"):]
    

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda" # the device to load the model onto

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True)

/home/leffff/PycharmProjects/travel-hack/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.79s/it]


In [6]:
import sys
sys.path.append("../")
from translation_service.translation import Translator

link = "https://01b3-109-252-98-213.ngrok-free.app"

translator = Translator(link)

In [11]:
browser = LLMBrowser(model, tokenizer, translator, max_new_tokens=200)
print(browser("Порекомендуй 3 парка в Москве?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


на основании предоставленного документа из Google Search, я рекомендую следующие три парка в Москве, Россия:

1. Парк Садовников Ю. М. Лужкова (Садовники Парк) - Этот парк расположен в Южном административном округе Москвы, в районе Нагатино-Садовники. Это хорошо сохранившаяся зеленая зона, охватывающая около 34,5 га. Адрес: Андроптерова пр-т, 58А, Москва, 115446 Часы работы: 24 часа Телефон: 8 (499) 175-33-69
2.Государственный исторический музей и его окрестности - Этот район не совсем парк, но это популярное место для посещения в Москве. В него входят Государственный исторический музей, Всероссийский выставочный центр, Большой Кремль


In [12]:
print(browser("Напиши стаью по теме: почему я должен пойти в Москвоский Планетарий"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


почему стоит посетить Московский планетарий: Космическое путешествие сквозь пространство и время

Введение:
Московский планетарий, основанный в 1929 году, является одной из самых популярных туристических достопримечательностей Москвы. Это знаковое учреждение предлагает посетителям увлекательный и просветительский опыт, позволяющий им исследовать чудеса Вселенной в захватывающей среде. В этой статье мы обсудим причины, по которым вы должны планировать посещение Московского планетария, опираясь на различные источники, чтобы предоставить вам информативную и убедительную перспективу.

1. Богатая история:
Основанный в 1929 году Московский планетарий имеет богатую историю, которая охватывает более девяти десятилетий. Это уникальное учреждение имеет легендарную историю и вдохновляет поколения людей смотреть на ночное небо с удивлением и любопытством. Посещая Московский Планетарий, вы станете частью непреходящей традиции


In [9]:
print(browser("Составь план туристической прогулки по Москве?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


чтобы создать туристический план для Москвы на основе предоставленного документа, вот предлагаемый маршрут для изучения города:

День 1:
1. Посетите Красную площадь и собор Святого Василия.
2.Исследуйте Государственный исторический музей и торговый центр Gum.
3. Прогуляйтесь по улице Арбат.
4.Насладитесь русским фольклорным представлением или балетом в Большом театре.

День 2:
1. Совершите экскурсию по Кремлю и его соборам, таким как Успенский собор и колокольня Ивана Великого.
2. Посетите Воробей Холм для панорамного вида на Москву.
3. Прогуляйтесь по очаровательному парку Зарядье.

День 3:
1. Посетите Третьяковскую галерею, чтобы увидеть русские иконы и искусство.
2. Исследуйте улицы Арцынского района


In [10]:
print(browser("Напиши статью на тему 5 главных трендов рынка корпоратиынх мероприятий в России?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


название: Пять заметных тенденций, формирующих ландшафт корпоративных событий в Москве, Россия

Москва, сердце России, известна своей богатой историей, яркой культурой и процветающей экономикой. Его шумная корпоративная сцена не является исключением, с постоянно развивающимся рынком корпоративных событий. В этой статье мы рассмотрим пять основных тенденций, формирующих этот динамичный сектор в Москве.

1. Виртуальные и гибридные события
Пандемия COVID-19 значительно повлияла на корпоративный ландшафт событий во всем мире, что привело к всплеску виртуальных и гибридных событий. Москва не исключение. Компании в Москве все чаще используют эти форматы для поддержания связи и взаимодействия со своими командами и клиентами, несмотря на физические ограничения. Виртуальные и гибридные мероприятия предлагают уникальные возможности для настройки, снижения затрат и расширения охвата, что делает их все более популярным трендом на рынке корпоративных мероприятий в Москве.

2. Экспериментальные собы